In [1]:
import pandas as pd
import numpy as np
import csv
import random

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.base import BaseEstimator
from scipy.optimize import minimize
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
from sknn.mlp import Classifier, Layer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler



#METHODS:
#['SLSQP', 'L-BFGS-B', 'CG', 'Newton-CG', 'TNC', 'BFGS', 'dogleg', 'trust-ncg']

#FUNCTIONS:
#objf_ens_optA_SLSQP
#objf_ens_optB_L-BFGS-B






Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:

def objf_ens_optA(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (12 in Airbnb competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optA, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA, x0, args=(Xs, y, self.n_class), 
                       method='SLSQP', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred
    
    

In [3]:
def objf_ens_optB(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 12
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optB, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB, x0, args=(Xs, y, self.n_class), 
                       method='L-BFGS-B', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred
    
    


In [4]:
print('Load data...')
DATA_DIR = "/Users/patrickkennedy/Desktop/Data_Science_MISC/Kaggle"
train = pd.read_csv(DATA_DIR + "/BNP_Paribas/train.csv")
test = pd.read_csv(DATA_DIR + "/BNP_Paribas/test.csv")

target = train['target'].values

train = train.drop(['ID','target'],axis=1)
id_test = test['ID'].values
test = test.drop(['ID'],axis=1)

print('Clearing...')
for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            #print "mean", train_series.mean()
            train.loc[train_series.isnull(), train_name] = -9999 #train_series.mean()
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = -9999 #train_series.mean()  #TODO
            

            


Load data...
Clearing...


In [10]:
#i can also think about initializing the split with a different random state too

n_classes = 2 
for_real = True

#this is what i'll change when i run the whole data set...
#essentially my train and test sets are already split

#Spliting data into train and test sets.
#X, X_test, y, y_test = train_test_split(train, target, test_size=0.2)
    
#Spliting train data into training and validation sets.
#X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25)

#print('Data shape:')
#print('X_train: %s, X_valid: %s, X_test: %s \n' %(X_train.shape, X_valid.shape, 
#                                                  X_test.shape))

#if for_real:
    #take the train, target and test data, and come up with a validation set from train
#    X_real = train
#    X_test_real = test
#    y_real = target
    
#    X_train_real, X_valid_real, y_train_real, y_valid_real = train_test_split(X_real, y_real, test_size=0.25)
    


In [33]:
#Defining the classifiers
#think about jittering the random state and then averaging the predictions together ...
#only good for extra trees, RF?, native XGB, NN
clfs = {#'LR'  : LogisticRegression(), 
        #'SVM' : SVC(probability=True, random_state=random_state), 
        #'RF'  : RandomForestClassifier(n_estimators=100, n_jobs=-1), 
        #'GBM' : GradientBoostingClassifier(n_estimators=50), 
        'ETC' : ExtraTreesClassifier(n_estimators=108, max_features=130, max_depth=12, n_jobs=-1),
        #'KNN' : KNeighborsClassifier(n_neighbors=5),
        'XGBc': XGBClassifier(objective='binary:logistic',
                              colsample_bytree=0.77638333498678636,
                              learning_rate=0.030567867858705199,
                              max_delta_step=4.6626180513766657,
                              min_child_weight=57.354121041109124,
                              n_estimators=478,
                              subsample=0.8069399976204783,
                              max_depth=6,
                              gamma=0.2966938071810209) #,
        #'NN'  : Pipeline([('min/max scaler', MinMaxScaler(feature_range=(-1.0, 1.0))),
        #                  ('neural network', Classifier(layers=[Layer("Rectifier", units=10),
        #                                                        Layer("Tanh", units=10),
        #                                                        Layer("Softmax")], 
        #                                                n_iter=5))])       
       }



#let's try best random without tuning and best random with tuning

In [12]:


#predictions on the validation and test sets
p_valid = []
p_test = []

p_valid_real = []
p_test_real = []
   
print('Performance of individual classifiers (1st layer) on X_test')   
print('------------------------------------------------------------')

#Spliting data into train and test sets.
X, X_test, y, y_test = train_test_split(train, target, test_size=0.2, random_state=117)
    
#Spliting train data into training and validation sets.
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=117)

X_real = train
X_test_real = test
y_real = target
    
X_train_real, X_valid_real, y_train_real, y_valid_real = train_test_split(X_real, y_real, test_size=0.25, 
                                                                          random_state=117)

random.seed(117)

for nm, clf in clfs.items():
    if nm == 'NN':
        #First run. Training on (X_train, y_train) and predicting on X_valid.
        clf.fit(X_train.as_matrix(), y_train)
        yv = clf.predict_proba(X_valid.as_matrix())
        p_valid.append(yv)
        
        #Second run. Training on (X, y) and predicting on X_test.
        clf.fit(X.as_matrix(), y)
        yt = clf.predict_proba(X_test.as_matrix())
        p_test.append(yt)
        
        if for_real:
            #First run. Training on (X_train, y_train) and predicting on X_valid.
            clf.fit(X_train_real.as_matrix(), y_train_real)
            yv_real = clf.predict_proba(X_valid_real.as_matrix())
            p_valid_real.append(yv_real)
        
            #Second run. Training on (X, y) and predicting on X_test.
            clf.fit(X_real.as_matrix(), y_real)
            yt_real = clf.predict_proba(X_test_real.as_matrix())
            p_test_real.append(yt_real)
    
    else:
        #First run. Training on (X_train, y_train) and predicting on X_valid.
        clf.fit(X_train, y_train)
        yv = clf.predict_proba(X_valid)
        p_valid.append(yv)
        
        #Second run. Training on (X, y) and predicting on X_test.
        clf.fit(X, y)
        yt = clf.predict_proba(X_test)
        p_test.append(yt)
        
        if for_real:
            #First run. Training on (X_train, y_train) and predicting on X_valid.
            clf.fit(X_train_real, y_train_real)
            yv_real = clf.predict_proba(X_valid_real)
            p_valid_real.append(yv_real)
        
            #Second run. Training on (X, y) and predicting on X_test.
            clf.fit(X_real, y_real)
            yt_real = clf.predict_proba(X_test_real)
            p_test_real.append(yt_real)
       
    #Printing out the performance of the classifier
    print('{:10s} {:2s} {:1.7f}'.format('%s: ' %(nm), 'logloss  =>', log_loss(y_test, yt)))
print('')


#when running the full data
#take out the logloss function... or alternatively, run both the split data and full data model so that
#i can compare my training logloss vs kaggle logloss


#maybe build a system that tries base models, shakes random states in order to find uncorrelated models?
#don't know why but my bayes_opt script is giving a worse score than untuned model (default params)

Performance of individual classifiers (1st layer) on X_test
------------------------------------------------------------
ETC:       logloss  => 0.4603708
XGBc:      logloss  => 0.4586818



In [38]:
%%time
print('MEAN MODELS: Performance of individual classifiers (1st layer) on X_test')   
print('------------------------------------------------------------------------')
p_valid_mean = []
p_test_mean = []

p_valid_mean_real = []
p_test_mean_real = []

best_model_score = 10.0

for nm, clf in clfs.items():
    p_valid_clf = []
    p_test_clf = []
    p_valid_clf_real = []
    p_test_clf_real = []
    
    holder = []
    
    for i in xrange(25):
    
        dummy = random.randint(1,10000)
        x = True
        while x == True:
            if dummy in holder:
                dummy = random.randint(1,10000)
            else:
                x = False
        holder.append(dummy)
    
        random.seed(dummy)
        
        #Spliting data into train and test sets.
        X, X_test, y, y_test = train_test_split(train, target, test_size=0.2, random_state=dummy)
    
        #Spliting train data into training and validation sets.
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=dummy)

        X_real = train
        X_test_real = test
        y_real = target
    
        X_train_real, X_valid_real, y_train_real, y_valid_real = train_test_split(X_real, y_real, test_size=0.25, 
                                                                                  random_state=dummy)
        
        
        if nm == 'NN':
            #First run. Training on (X_train, y_train) and predicting on X_valid.
            clf.fit(X_train.as_matrix(), y_train)
            yv = clf.predict_proba(X_valid.as_matrix())
            p_valid_clf.append(yv)
        
            #Second run. Training on (X, y) and predicting on X_test.
            clf.fit(X.as_matrix(), y)
            yt = clf.predict_proba(X_test.as_matrix())
            p_test_clf.append(yt)
        
            if for_real:
                #First run. Training on (X_train, y_train) and predicting on X_valid.
                clf.fit(X_train_real.as_matrix(), y_train_real)
                yv_real = clf.predict_proba(X_valid_real.as_matrix())
                p_valid_clf_real.append(yv_real)
        
                #Second run. Training on (X, y) and predicting on X_test.
                clf.fit(X_real.as_matrix(), y_real)
                yt_real = clf.predict_proba(X_test_real.as_matrix())
                p_test_clf_real.append(yt_real)
    
        else:
            #First run. Training on (X_train, y_train) and predicting on X_valid.
            clf.fit(X_train, y_train)
            yv = clf.predict_proba(X_valid)
            p_valid_clf.append(yv)
        
            #Second run. Training on (X, y) and predicting on X_test.
            clf.fit(X, y)
            yt = clf.predict_proba(X_test)
            p_test_clf.append(yt)
        
            #if log_loss(y_test, yt) < best_model_score:
            #    best_model_score = log_loss(y_test, yt)
            #    best_dummy = dummy
            #    print nm, i, best_model_score, best_dummy
            #else:
            #    print nm, i
        
            if for_real:
                #First run. Training on (X_train, y_train) and predicting on X_valid.
                clf.fit(X_train_real, y_train_real)
                yv_real = clf.predict_proba(X_valid_real)
                p_valid_clf_real.append(yv_real)
        
                #Second run. Training on (X, y) and predicting on X_test.
                clf.fit(X_real, y_real)
                yt_real = clf.predict_proba(X_test_real)
                p_test_clf_real.append(yt_real)
            
       
    #Printing out the performance of the classifier
    mean_pred_cv = np.mean(p_valid_clf, axis=0)
    mean_pred_test = np.mean(p_test_clf, axis=0)
    p_valid_mean.append(mean_pred_cv)
    p_test_mean.append(mean_pred_test)
    
    mean_real_pred_cv = np.mean(p_valid_clf_real, axis=0)
    mean_real_pred_test = np.mean(p_test_clf_real, axis=0)
    p_valid_mean_real.append(mean_real_pred_cv)
    p_test_mean_real.append(mean_real_pred_test)
    
    
    #i take the predictions from each of the models of each iteration through the random states
    #and then i will run logloss on each set of preds and take the best models? best uncorrelated models?
    #if nm == 'ETC':
    #    predsETC_valid = p_valid_clf
    #    predsETC_test = p_test_clf
    #    predsETC_valid_real = p_valid_clf_real
    #    predsETC_test_real = p_test_clf_real
    if nm == 'XGBc':
        predsXGBc_valid = p_valid_clf
        predsXGBc_test = p_test_clf
        predsXGBc_valid_real = p_valid_clf_real
        predsXGBc_test_real = p_test_clf_real
    elif nm == 'NN':
        predsNN_valid = p_valid_clf
        predsNN_test = p_test_clf
        predsNN_valid_real = p_valid_clf_real
        predsNN_test_real = p_test_clf_real
        
        
    print('{:10s} {:2s} {:1.7f}'.format('%s - mean: ' %(nm), 'logloss  =>', log_loss(y_test, mean_pred_test)))
    

        
        
    
print('')

#also try setting different parameters for the XGB and add a NN to the mix
#either use bayesopt for each classifier and putting those into this model, -or- 
#randomize both parameters and by random_state... i could do several loops here
#lots of comp time but each random_state run a series of different parameters and
#take the average result for that particular random_state, then run the same
#parameters on the next random_state (don't want random X random as that is hard to replicate)...
#too many combos... let's do bayes_opt

#Without Bayes_Opt
#ETC 9 0.466434680757 9958
#ETC 63 0.464552567051 117
#ETC 242 0.464125730276 5299

#XGBc 17 0.465919276422 6384
#XGBc 51 0.46307720874 9958


#With Bayes_Opt
#ETC 3 0.461308763189 8830
#ETC 14 0.460268248358 117
#cancelled early



#think about testing particular random states across models... like 117 but for xgbclassifier
#just using 117 overfits bad

#how about 117, 9958, 8830


#next look at the results and kludge together models with relevant random states / params (if needed)
#filter through real data so i can finish the algo



#also think about running a long stretch of code where correlations are made between models and the two models
#with the lowest correlations are kept... but what about multiple that have low correlations with one another?
#maybe this is a future project...

#at the least i can correlate the two models? why would this matter? how do those optimizers work?

MEAN MODELS: Performance of individual classifiers (1st layer) on X_test
------------------------------------------------------------------------
ETC - mean:  logloss  => 0.5442996


KeyboardInterrupt: 

In [37]:
#predsETC_valid -- run correlations on each and see which ones are least correlated


250

In [40]:
#pd.DataFrame({"p_valid_0_0": p_valid[0][:,0],
#              "p_valid_0_1": p_valid[0][:,1],
#              "p_valid_1_0": p_valid[1][:,0],
#              "p_valid_1_1": p_valid[1][:,1],
#             }).to_csv('p_valid_preds_for_EXT_and_XGBc_single_models.csv',index=False)

#pd.DataFrame({"p_test_0_0" : p_test[0][:,0],
#              "p_test_0_1" : p_test[0][:,1],
#              "p_test_1_0" : p_test[1][:,0],
#              "p_test_1_1" : p_test[1][:,1]
#             }).to_csv('p_test_preds_for_EXT_and_XGBc_single_models.csv',index=False)

#pd.DataFrame({"p_valid_real_0_0": p_valid_real[0][:,0],
#              "p_valid_real_0_1": p_valid_real[0][:,1],
#              "p_valid_real_1_0": p_valid_real[1][:,0],
#              "p_valid_real_1_1": p_valid_real[1][:,1]
#             }).to_csv('p_valid_real_preds_for_EXT_and_XGBc_single_models.csv',index=False)

#pd.DataFrame({"p_test_real_0_0" : p_test_real[0][:,0],
#              "p_test_real_0_1" : p_test_real[0][:,1],
#              "p_test_real_1_0" : p_test_real[1][:,0],
#              "p_test_real_1_1" : p_test_real[1][:,1]
#             }).to_csv('p_test_real_preds_for_EXT_and_XGBc_single_models.csv',index=False)


In [49]:
#p_valid = pd.read_csv("p_valid_preds_for_EXT_and_XGBc_single_models.csv")
#p_test = pd.read_csv("p_test_preds_for_EXT_and_XGBc_single_models.csv")


In [143]:
for_real = True

In [25]:
print('Performance of optimization based ensemblers (2nd layer) on X_test')   
print('------------------------------------------------------------')
    
#Creating the data for the 2nd layer.
XV = np.hstack(p_valid)
XT = np.hstack(p_test)  


#XV = p_valid.as_matrix()
#XT = p_test.as_matrix()

n_classes = 2

 
method = ['SLSQP',
          'L-BFGS-B',
          #'CG',
          #'Newton-CG',
          #'TNC',
          #'BFGS',
          #'dogleg',
          #'trust-ncg',
          #'Nelder-Mead',
          #'Powell'
          #'COBYLA'
         ]         
          
y_enA = []
y_enB = []
y_ccA = []
y_ccB = []


classesA = [EN_optA_SLSQP(n_classes), 
           EN_optA_L_BFGS_B(n_classes)#,
           #EN_optA_CG(n_classes),
           #EN_optA_Newton_CG(n_classes),
           #EN_optA_TNC(n_classes),
           #EN_optA_BFGS(n_classes),
           #EN_optA_dogleg(n_classes),
           #EN_optA_trust_ncg(n_classes),
           #EN_optA_Nelder_Mead(n_classes),
           #EN_optA_Powell(n_classes)
           #EN_optA_COBYLA(n_classes)
            ]

classesB = [EN_optB_SLSQP(n_classes), 
           EN_optB_L_BFGS_B(n_classes)#,
           #EN_optB_CG(n_classes),
           #EN_optB_Newton_CG(n_classes),
           #EN_optB_TNC(n_classes),
           #EN_optB_BFGS(n_classes),
           #EN_optB_dogleg(n_classes),
           #EN_optB_trust_ncg(n_classes)
           #EN_optB_Nelder_Mead(n_classes),
           #EN_optB_Powell(n_classes)
           #EN_optB_COBYLA(n_classes)
           ]

for c, m in zip(classesA, method):
    enA = c
    enA.fit(XV, y_valid)
    w_enA = enA.w
    y_enA.append(enA.predict_proba(XT))
    print('{:20s} {:2s} {:1.7f}'.format('EN_optA_'+m, 'logloss  =>', log_loss(y_test, y_enA[-1])))
    
    #Calibrated version of EN_optA_SLSQP 
    cc_optA = CalibratedClassifierCV(enA, method='isotonic', cv=10)
    cc_optA.fit(XV, y_valid)
    y_ccA.append(cc_optA.predict_proba(XT))
    print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optA', 'logloss  =>', log_loss(y_test, y_ccA[-1])))
    

for c, m in zip(classesB, method):
    enB = c
    enB.fit(XV, y_valid)
    w_enB = enB.w
    y_enB.append(enB.predict_proba(XT))
    print('{:20s} {:2s} {:1.7f}'.format('EN_optB_'+m, 'logloss  =>', log_loss(y_test, y_enB[-1])))
    
    #Calibrated version of EN_optA_SLSQP 
    cc_optB = CalibratedClassifierCV(enB, method='isotonic', cv=10)
    cc_optB.fit(XV, y_valid)
    y_ccB.append(cc_optB.predict_proba(XT))
    print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optB', 'logloss  =>', log_loss(y_test, y_ccB[-1])))

#enA = EN_optA_SLSQP(n_classes)
#enA.fit(XV, y_valid)
#w_enA = enA.w
#y_enA = enA.predict_proba(XT)
#print('{:20s} {:2s} {:1.7f}'.format('EN_optA:', 'logloss  =>', log_loss(y_test, y_enA)))
    

#EN_optA_SLSQP
#enA = EN_optA(n_classes)
#enA.fit(XV, y_valid)
#w_enA = enA.w
#y_enA = enA.predict_proba(XT)
#print('{:20s} {:2s} {:1.7f}'.format('EN_optA:', 'logloss  =>', log_loss(y_test, y_enA)))
    
#Calibrated version of EN_optA_SLSQP 
#cc_optA = CalibratedClassifierCV(enA, method='isotonic', cv=10)
#cc_optA.fit(XV, y_valid)
#y_ccA = cc_optA.predict_proba(XT)
#print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optA:', 'logloss  =>', log_loss(y_test, y_ccA)))


#EN_optB
#enB = EN_optB(n_classes) 
#enB.fit(XV, y_valid)
#w_enB = enB.w
#y_enB = enB.predict_proba(XT)
#print('{:20s} {:2s} {:1.7f}'.format('EN_optB:', 'logloss  =>', log_loss(y_test, y_enB)))

#Calibrated version of EN_optB
#cc_optB = CalibratedClassifierCV(enB, method='isotonic', cv=10)
#cc_optB.fit(XV, y_valid)
#y_ccB = cc_optB.predict_proba(XT)
#print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optB:', 'logloss  =>', log_loss(y_test, y_ccB)))
#print('')



if for_real:
    print('REAL: Performance of optimization based ensemblers (2nd layer) on X_test')   
    print('------------------------------------------------------------')
    
    #Creating the data for the 2nd layer.
    XV_real = np.hstack(p_valid_real)
    XT_real = np.hstack(p_test_real)  
    
    #XV_real = p_valid_real.as_matrix()
    #XT_real = p_test_real.as_matrix()
    
    n_classes = 2
    
    y_enA_real = []
    y_ccA_real = []
    y_enB_real = []
    y_ccB_real = []
    
    for c, m in zip(classesA, method):
        enA_real = c
        enA_real.fit(XV_real, y_valid_real)
        w_enA_real = enA_real.w
        y_enA_real.append(enA_real.predict_proba(XT_real))
    
        #Calibrated version of EN_optA_SLSQP 
        cc_optA_real = CalibratedClassifierCV(enA_real, method='isotonic', cv=10)
        cc_optA_real.fit(XV_real, y_valid_real)
        y_ccA_real.append(cc_optA_real.predict_proba(XT_real))
    

    for c, m in zip(classesB, method):
        enB_real = c
        enB_real.fit(XV_real, y_valid_real)
        w_enB_real = enB_real.w
        y_enB_real.append(enB_real.predict_proba(XT_real))
    
        #Calibrated version of EN_optA_SLSQP 
        cc_optB_real = CalibratedClassifierCV(enB_real, method='isotonic', cv=10)
        cc_optB_real.fit(XV_real, y_valid_real)
        y_ccB_real.append(cc_optB_real.predict_proba(XT_real))
    
    
    
    
    
    
    #EN_optA
    #enA_real = EN_optA(n_classes)
    #enA_real.fit(XV_real, y_valid_real)
    #w_enA_real = enA_real.w
    #y_enA_real = enA_real.predict_proba(XT_real)
    #print('{:20s} {:2s} {:1.7f}'.format('EN_optA:', 'logloss  =>', log_loss(y_test, y_enA)))
    
    #Calibrated version of EN_optA 
    #cc_optA_real = CalibratedClassifierCV(enA_real, method='isotonic')
    #cc_optA_real.fit(XV_real, y_valid_real)
    #y_ccA_real = cc_optA_real.predict_proba(XT_real)
    #print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optA:', 'logloss  =>', log_loss(y_test, y_ccA)))
        
    #EN_optB
    #enB_real = EN_optB(n_classes) 
    #enB_real.fit(XV_real, y_valid_real)
    #w_enB_real = enB_real.w
    #y_enB_real = enB_real.predict_proba(XT_real)
    #print('{:20s} {:2s} {:1.7f}'.format('EN_optB:', 'logloss  =>', log_loss(y_test, y_enB)))

    #Calibrated version of EN_optB
    #cc_optB_real = CalibratedClassifierCV(enB_real, method='isotonic')
    #cc_optB_real.fit(XV_real, y_valid_real)
    #y_ccB_real = cc_optB_real.predict_proba(XT_real)
    #print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optB:', 'logloss  =>', log_loss(y_test, y_ccB)))
    #print('')
    
    
    


Performance of optimization based ensemblers (2nd layer) on X_test
------------------------------------------------------------
EN_optA_SLSQP        logloss  => 0.4575958
Calibrated_EN_optA   logloss  => 0.4566262
EN_optA_L-BFGS-B     logloss  => 0.4575340


/Users/patrickkennedy/anaconda/lib/python2.7/site-packages/scipy/optimize/_minimize.py:397: RuntimeWarning: Method L-BFGS-B cannot handle constraints.
  RuntimeWarning)


Calibrated_EN_optA   logloss  => 0.4566628
EN_optB_SLSQP        logloss  => 0.4575958
Calibrated_EN_optB   logloss  => 0.4566262
EN_optB_L-BFGS-B     logloss  => 0.4576971
Calibrated_EN_optB   logloss  => 0.4569401
REAL: Performance of optimization based ensemblers (2nd layer) on X_test
------------------------------------------------------------


In [30]:
print('MEAN:  Performance of optimization based ensemblers (2nd layer) on X_test')   
print('------------------------------------------------------------')
    
#Creating the data for the 2nd layer.
XV_mean = np.hstack(p_valid_mean)
XT_mean = np.hstack(p_test_mean)  
        
#EN_optA
enA_mean = EN_optA(n_classes)
enA_mean.fit(XV_mean, y_valid)
w_enA_mean = enA_mean.w
y_enA_mean = enA_mean.predict_proba(XT_mean)
print('{:20s} {:2s} {:1.7f}'.format('EN_optA:', 'logloss  =>', log_loss(y_test, y_enA_mean)))
    
#Calibrated version of EN_optA 
cc_optA_mean = CalibratedClassifierCV(enA_mean, method='isotonic')
cc_optA_mean.fit(XV_mean, y_valid)
y_ccA_mean = cc_optA_mean.predict_proba(XT_mean)
print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optA:', 'logloss  =>', log_loss(y_test, y_ccA_mean)))
        
#EN_optB
enB_mean = EN_optB(n_classes) 
enB_mean.fit(XV_mean, y_valid)
w_enB_mean = enB_mean.w
y_enB_mean = enB_mean.predict_proba(XT_mean)
print('{:20s} {:2s} {:1.7f}'.format('EN_optB:', 'logloss  =>', log_loss(y_test, y_enB_mean)))

#Calibrated version of EN_optB
cc_optB_mean = CalibratedClassifierCV(enB_mean, method='isotonic')
cc_optB_mean.fit(XV_mean, y_valid)
y_ccB_mean = cc_optB_mean.predict_proba(XT_mean)  
print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optB:', 'logloss  =>', log_loss(y_test, y_ccB_mean)))
print('')

if for_real:
    print('MEAN:  Performance of optimization based ensemblers (2nd layer) on X_test')   
    print('------------------------------------------------------------')
    
    #Creating the data for the 2nd layer.
    XV_mean_real = np.hstack(p_valid_mean_real)
    XT_mean_real = np.hstack(p_test_mean_real)  
        
    #EN_optA
    enA_mean_real = EN_optA(n_classes)
    enA_mean_real.fit(XV_mean_real, y_valid_real)
    w_enA_mean_real = enA_mean_real.w
    y_enA_mean_real = enA_mean_real.predict_proba(XT_mean_real)
    #print('{:20s} {:2s} {:1.7f}'.format('EN_optA:', 'logloss  =>', log_loss(y_test, y_enA_mean)))
    
    #Calibrated version of EN_optA 
    cc_optA_mean_real = CalibratedClassifierCV(enA_mean_real, method='isotonic')
    cc_optA_mean_real.fit(XV_mean_real, y_valid_real)
    y_ccA_mean_real = cc_optA_mean_real.predict_proba(XT_mean_real)
    #print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optA:', 'logloss  =>', log_loss(y_test, y_ccA_mean)))
        
    #EN_optB
    enB_mean_real = EN_optB(n_classes) 
    enB_mean_real.fit(XV_mean_real, y_valid_real)
    w_enB_mean_real = enB_mean_real.w
    y_enB_mean_real = enB_mean_real.predict_proba(XT_mean_real)
    #print('{:20s} {:2s} {:1.7f}'.format('EN_optB:', 'logloss  =>', log_loss(y_test, y_enB_mean)))

    #Calibrated version of EN_optB
    cc_optB_mean_real = CalibratedClassifierCV(enB_mean_real, method='isotonic')
    cc_optB_mean_real.fit(XV_mean_real, y_valid_real)
    y_ccB_mean_real = cc_optB_mean_real.predict_proba(XT_mean_real)  
    #print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optB:', 'logloss  =>', log_loss(y_test, y_ccB_mean)))
    print('')

MEAN:  Performance of optimization based ensemblers (2nd layer) on X_test
------------------------------------------------------------
EN_optA:             logloss  => 0.5162222
Calibrated_EN_optA:  logloss  => 0.5162544
EN_optB:             logloss  => 0.5161750
Calibrated_EN_optB:  logloss  => 0.5162259

MEAN:  Performance of optimization based ensemblers (2nd layer) on X_test
------------------------------------------------------------



In [182]:
from scipy.stats.stats import pearsonr   

slsqpA = y_enA[0][:,1]
lbfgsbA = y_enA[1][:,1]
tncA = y_enA[2][:,1]
bfgsA = y_enA[3][:,1]
neldermeadA = y_enA[4][:,1]
powellA = y_enA[5][:,1]

slsqpB = y_enB[0][:,1]
lbfgsbB = y_enB[1][:,1]
tncB = y_enB[2][:,1]
bfgsB = y_enB[3][:,1]
neldermeadB = y_enB[4][:,1]
powellB = y_enB[5][:,1]


cc_slsqpA = y_ccA[0][:,1]
cc_lbfgsbA = y_ccA[1][:,1]
cc_tncA = y_ccA[2][:,1]
cc_bfgsA = y_ccA[3][:,1]
cc_neldermeadA = y_ccA[4][:,1]
cc_powellA = y_ccA[5][:,1]

cc_slsqpB = y_ccB[0][:,1]
cc_lbfgsbB = y_ccB[1][:,1]
cc_tncB = y_ccB[2][:,1]
cc_bfgsB = y_ccB[3][:,1]
cc_neldermeadB = y_ccB[4][:,1]
cc_powellB = y_ccB[5][:,1]

opt_results = pd.DataFrame({"SLSQP_A":slsqpA, 
                            "L_BFGS_B_A":lbfgsbA, 
                            "TNC_A":tncA,
                            "BFGS_A":bfgsA, 
                            "Nelder_Mead_A":neldermeadA,
                            "Powell_A":powellA,
                            "SLSQP_B":slsqpB, 
                            "L_BFGS_B_B":lbfgsbB, 
                            "TNC_B":tncB,
                            "BFGS_B":bfgsB, 
                            "Nelder_Mead_B":neldermeadB,
                            "Powell_B":powellB,
                            "cc_SLSQP_A":cc_slsqpA, 
                            "cc_L_BFGS_B_A":cc_lbfgsbA, 
                            "cc_TNC_A":cc_tncA,
                            "cc_BFGS_A":cc_bfgsA, 
                            "cc_Nelder_Mead_A":cc_neldermeadA,
                            "cc_Powell_A":cc_powellA,
                            "cc_SLSQP_B":cc_slsqpB, 
                            "cc_L_BFGS_B_B":cc_lbfgsbB, 
                            "cc_TNC_B":cc_tncB,
                            "cc_BFGS_B":cc_bfgsB, 
                            "cc_Nelder_Mead_B":cc_neldermeadB,
                            "cc_Powell_B":cc_powellB})
            

opt_results.corr()






#any of these uncorrelated?? maybe not... makes the next part a bit more difficult
 
#method = ['SLSQP',
#          'L-BFGS-B',
          #'CG',
          #'Newton-CG',
#          'TNC',
#          'BFGS',
          #'dogleg',
          #'trust-ncg',
#          'Nelder-Mead',
#          'Powell'
          #'COBYLA'
#         ]   

In [193]:
y_enA_total = np.mean(y_enA, axis=0)
y_ccA_total = np.mean(y_ccA, axis=0)
y_enB_total = np.mean(y_enB, axis=0)
y_ccB_total = np.mean(y_ccB, axis=0)

#y_enA_real_total = np.mean(y_enA_real, axis=0)
#y_ccA_real_total = np.mean(y_ccA_real, axis=0)
#y_enB_real_total = np.mean(y_enB_real, axis=0)
#y_ccB_real_total = np.mean(y_ccB_real, axis=0)

In [31]:
#come up with better weights here... reflect that in calibration performance
#y_3l = (y_enA_total * 2./9.) + (y_ccA_total * 4./9.) + (y_enB_total * 2./9.) + (y_ccB_total * 1./9.)
#print('{:20s} {:2s} {:1.7f}'.format('3rd_layer:', 'logloss  =>', log_loss(y_test, y_3l)))


#optimize weighting of the 3rd level - keep the same weighting for real data
best_score = 10.0

for i in range(10000):
    first = random.randint(0,20)
    second = random.randint(0,20)
    third = random.randint(0,20)
    fourth = random.randint(0,20)
    total = first + second + third + fourth
    first = first / (total * 1.0)
    second = second / (total * 1.0)
    third = third / (total * 1.0)
    fourth = fourth / (total * 1.0)
    
    y_3l = (y_enA_mean * first) + (y_ccA_mean * second) + (y_enB_mean * third) + (y_ccB_mean * fourth)
    current_score = log_loss(y_test, y_3l)
    
    if current_score < best_score:
        print('{:20s} {:2s} {:1.7f}'.format('3rd_layer:', 'logloss  =>', log_loss(y_test, y_3l)))
        #print first, second, third, fourth
        best_score = current_score
        best_first = first
        best_second = second
        best_third = third
        best_fourth = fourth

3rd_layer:           logloss  => 0.5160226
3rd_layer:           logloss  => 0.5160198
3rd_layer:           logloss  => 0.5160193
3rd_layer:           logloss  => 0.5160189
3rd_layer:           logloss  => 0.5160188


22865

In [32]:
if for_real:
    preds = (y_enA_mean_real * best_first) + \
            (y_ccA_mean_real * best_second) + \
            (y_enB_mean_real * best_third) + \
            (y_ccB_mean_real * best_fourth)
            
else:
    preds = (y_enA_total * best_first) + \
            (y_ccA_total * best_second) + \
            (y_enB_total * best_third) + \
            (y_ccB_total * best_fourth)
            
pd.DataFrame({"ID": id_test, "PredictedProb": preds[:,1]}).to_csv('3-level_calibrated_model_XGBc_ET__picked3randomstate.csv',index=False)

In [ ]:
#.46631 -- mean of min methods
#.46591 -- slsqp for A, l-bfgs-b for B

#non-bayes-opt
#.45256 -- mean of min methods
#.45233 -- slsqp for A, l-bfgs-b for B

#kNN, NN, XGBc, ET, RF
#.45554 / train: .46515



pd.DataFrame({"ID": id_test, "PredictedProb": preds[:,1]}).to_csv('3-level_calibrated_model_bayesopt.csv',index=False)

In [ ]:
#METHODS:

#‘Nelder-Mead’
#‘Powell’
#‘CG’
#‘BFGS’
#‘Newton-CG’
#‘Anneal (deprecated as of scipy version 0.14.0)’
#‘L-BFGS-B’
#‘TNC’
#‘COBYLA’
#‘SLSQP’
#‘dogleg’
#‘trust-ncg’


#FUNCTIONS:
#objf_ens_optA_SLSQP
#objf_ens_optB_L-BFGS-B


In [14]:
#COBYLA
def objf_ens_optA_COBYLA(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (12 in Airbnb competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA_COBYLA(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optA_COBYLA, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA_COBYLA, x0, args=(Xs, y, self.n_class), 
                       method='COBYLA', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred
    

def objf_ens_optB_COBYLA(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 12
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB_COBYLA(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optB_COBYLA, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB_COBYLA, x0, args=(Xs, y, self.n_class), 
                       method='COBYLA', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred
    

In [15]:
#Powell
def objf_ens_optA_Powell(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (12 in Airbnb competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA_Powell(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optA_Powell, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA_Powell, x0, args=(Xs, y, self.n_class), 
                       method='Powell', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred
    

def objf_ens_optB_Powell(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 12
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB_Powell(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optB_Powell, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB_Powell, x0, args=(Xs, y, self.n_class), 
                       method='Powell', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred
    

In [16]:
#Nelder-Mead
def objf_ens_optA_Nelder_Mead(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (12 in Airbnb competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA_Nelder_Mead(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optA_Nelder_Mead, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA_Nelder_Mead, x0, args=(Xs, y, self.n_class), 
                       method='Nelder-Mead', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred
    

def objf_ens_optB_Nelder_Mead(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 12
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB_Nelder_Mead(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optB_Nelder_Mead, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB_Nelder_Mead, x0, args=(Xs, y, self.n_class), 
                       method='Nelder-Mead', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred
    

In [17]:
#SLSQP
def objf_ens_optA_SLSQP(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (12 in Airbnb competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA_SLSQP(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optA_SLSQP, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA_SLSQP, x0, args=(Xs, y, self.n_class), 
                       method='SLSQP', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred
    

def objf_ens_optB_SLSQP(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 12
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB_SLSQP(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optB_SLSQP, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB_SLSQP, x0, args=(Xs, y, self.n_class), 
                       method='SLSQP', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred
    

In [18]:
#L-BFGS-B
def objf_ens_optA_L_BFGS_B(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (12 in Airbnb competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA_L_BFGS_B(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optA_L_BFGS_B, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA_L_BFGS_B, x0, args=(Xs, y, self.n_class), 
                       method='L-BFGS-B', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred
    

def objf_ens_optB_L_BFGS_B(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 12
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB_L_BFGS_B(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optB_L_BFGS_B, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB_L_BFGS_B, x0, args=(Xs, y, self.n_class), 
                       method='L-BFGS-B', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred
    


In [19]:
#CG
def objf_ens_optA_CG(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (12 in Airbnb competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA_CG(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optA_CG, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA_CG, x0, args=(Xs, y, self.n_class), 
                       method='CG', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred
    

def objf_ens_optB_CG(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 12
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB_CG(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optB_CG, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB_CG, x0, args=(Xs, y, self.n_class), 
                       method='CG', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred
    


In [20]:
#Newton-CG
def objf_ens_optA_Newton_CG(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (12 in Airbnb competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA_Newton_CG(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optA_Newton_CG, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA_Newton_CG, x0, args=(Xs, y, self.n_class), 
                       method='Newton-CG', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred
    

def objf_ens_optB_Newton_CG(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 12
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB_Newton_CG(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optB_Newton_CG, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB_Newton_CG, x0, args=(Xs, y, self.n_class), 
                       method='Newton-CG', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred
    


In [21]:
#TNC 
def objf_ens_optA_TNC(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (12 in Airbnb competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA_TNC(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optA_TNC, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA_TNC, x0, args=(Xs, y, self.n_class), 
                       method='TNC', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred
    

def objf_ens_optB_TNC(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 12
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB_TNC(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optB_TNC, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB_TNC, x0, args=(Xs, y, self.n_class), 
                       method='TNC', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred
    


In [22]:
#BFGS
def objf_ens_optA_BFGS(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (12 in Airbnb competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA_BFGS(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optA_BFGS, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA_BFGS, x0, args=(Xs, y, self.n_class), 
                       method='BFGS', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred
    

def objf_ens_optB_BFGS(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 12
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB_BFGS(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optB_BFGS, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB_BFGS, x0, args=(Xs, y, self.n_class), 
                       method='BFGS', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred
    


In [23]:
#dogleg
def objf_ens_optA_dogleg(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (12 in Airbnb competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA_dogleg(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optA_dogleg, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA_dogleg, x0, args=(Xs, y, self.n_class), 
                       method='dogleg', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred
    

def objf_ens_optB_dogleg(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 12
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB_dogleg(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optB_dogleg, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB_dogleg, x0, args=(Xs, y, self.n_class), 
                       method='dogleg', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred
    


In [24]:
#trust-ncg
def objf_ens_optA_trust_ncg(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (12 in Airbnb competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA_trust_ncg(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optA_trust_ncg, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA_trust_ncg, x0, args=(Xs, y, self.n_class), 
                       method='trust_ncg', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred
    

def objf_ens_optB_trust_ncg(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 12
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB_trust_ncg(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optB_trust_ncg, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB_trust_ncg, x0, args=(Xs, y, self.n_class), 
                       method='trust_ncg', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred
    
